# Chat Completions API

Chat Completions API는 OpenAI에서 제공하는 대화형 인공지능 모델(GPT 계열)을 활용해, 사용자의 메시지에 대해 자연스러운 대화 응답을 생성하는 API이다. 이 API는 챗봇, AI 비서, 자동화된 상담 시스템 등 다양한 대화형 서비스에 적용할 수 있다.


- **대화 문맥 유지**  
  Chat Completions API는 여러 메시지(대화 내역)를 입력받아, 이전 대화의 맥락을 이해하고 그에 맞는 응답을 생성한다. 즉, 단순히 한 문장만을 이어 쓰는 것이 아니라, 대화의 흐름을 반영하여 자연스러운 대화를 이어갈 수 있다.

- **역할(Role) 기반 메시지 구조**  
  입력 메시지는 배열 형태로 전달하며, 각 메시지는 `role`과 `content`로 구성된다.  
  - `system`: AI의 태도, 성격, 역할을 정의(예: "너는 친절한 도우미야.")
  - `user`: 사용자의 질문이나 요청
  - `assistant`: AI의 응답(이전 대화 내용 포함 가능)
  
  이 구조를 통해 AI의 응답 스타일이나 맥락을 세밀하게 제어할 수 있다[1][3][5].

**주요 파라미터 설명**

| 파라미터        | 설명                                                                 |
|----------------|----------------------------------------------------------------------|
| model          | 사용할 언어 모델명 (예: gpt-3.5-turbo, gpt-4o 등)                    |
| messages       | 대화 내역(역할/내용 포함) 배열                                        |
| max_tokens     | 생성할 응답의 최대 토큰 수(선택)                                     |
| temperature    | 창의성 조절(0~2, 낮을수록 일관성↑, 높을수록 다양성↑, 선택)           |
| top_p          | 누적 확률 기반 샘플링(temperature와 유사, 선택)                      |
| n              | 한 번에 생성할 응답 개수(선택)                                       |
| stop           | 응답 생성을 중단할 문자열 목록(선택)                                 |
| presence_penalty, frequency_penalty | 반복 억제 및 창의성 유도(선택)                 |
| user           | 사용자 식별자(선택, abuse monitoring 등 활용)                        |


- 위 예시에서 `messages` 배열에는 대화의 모든 메시지가 순서대로 들어가야 한다.  
- OpenAI는 이전 요청을 기억하지 않기 때문에, 매 API 호출마다 대화 내역 전체를 함께 보내야 한다.

In [2]:
from google.colab import userdata
import os
from openai import OpenAI

# OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
# client = OpenAI(api_key=OPENAI_API_KEY)

# 환경변수 지정
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
client = OpenAI()

### 대화형 챗봇

In [ ]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    message=[
        {'role': 'system','content': '너는 친절한 챗봇입니다.'},
        {'role':'user','content': '안녕,내 이름은 차은우야~'}
    ]
)
print(response.choices[0].message.content)

## 반복처리

In [ ]:
# 대화내역을 로깅
messages = [
    {'role': 'system','content': '너는 친절한 챗봇이다.'}
]

print('종료하려면,exit를 입력하세요...')
while True:
  # 사용자 입력
  user_input = input('User:')

  if user_input.strip().lower() == 'exit':
    print('채팅을 종료합니다...')
    break

  # messages에 사용자 입력 추가
  messages.append({'role':'user','content': user_input})
  print(messages)

  # LLM 요청
  response = client.chat.completions.create(
      model='gpt-4.1',
      messages=messages,
      temperature=1
  )
  assistant_message = response.choices[0].message.content
  print(f'Assistant:'{assistant_message})

  # messages 챗봇 출력 추가
  messages.apeend({'role':'assistant','content': assistant_message})

## stream

In [ ]:
client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{'role':'user','content':'지금 stream테스트할거야,아주 좋아.'}]
    stream=True
)
for chunk in stream:
  content = chunk.choices[0].delta.content
  if content is not None:
    print(chunk.choices[0].delta.content,end=' ')

In [ ]:
# 스트림 버전
# 대화내역을 로깅
messages = [
    {'role': 'system','content': '너는 친절한 챗봇이다.'}
]

print('종료하려면,exit를 입력하세요...')
while True:
  # 사용자 입력
  user_input = input('User:')

  if user_input.strip().lower() == 'exit':
    print('채팅을 종료합니다...')
    break

  # messages에 사용자 입력 추가
  messages.append({'role':'user','content': user_input})
  print(messages)

  # LLM 요청
  response = client.chat.completions.create(
      model='gpt-4.1',
      messages=messages,
      temperature=1
  )
  assistant_message = response.choices[0].message.content
  print(f'Assistant:'{assistant_message})

  # messages 챗봇 출력 추가
  messages.apeend({'role':'assistant','content': assistant_message})

# Token counting

비용 = ((입출력 토큰수 * 단가) + (출력 토큰수 * 단가)) * 월 서비스 호출수

In [4]:
!pip install tiktoken

In [5]:
# 각 모델에 따른 토커나이저(인코딩) 가져오기
import tiktoken

gpt35 = tiktoken.encoding_for_model('gpt-3.5')
gpt4o = tiktoken.encoding_for_model('gpt-4o')
gpt41 = tiktoken.get_encoding('cl100k_base')

print(gpt35)
print(gpt4o)

<Encoding 'cl100k_base'>
<Encoding 'o200k_base'>


In [6]:
# 토큰수 세기
encoded_gpt35 = gpt35.encode('아버지가 방에 들어가신다.')
encode_gpt4o = gpt4o.encode('아버지가 방에 들어가신다.')

print(len(encoded_gpt35))
print(len(encode_gpt4o))

13
10


### 토큰비용 계산하기

In [ ]:
text = """
더불어민주당이 6일 한동훈 국민의힘 대표가 윤석열 대통령 탄핵에 사실상 찬성 입장을 시사하자 7일로 예정됐던 윤석열 대통령의 탄핵소추안 표결을 앞당기는 방안을 고심하고 있다. 이재명 민주당 대표는 “한 대표의 입장을 정확하게 파악하는 것이 우선”이라며 신중한 태도를 보였다.

이 대표는 이날 기자들을 만나 윤 대통령 탄핵 표결을 앞당기는 방안에 대해 “지금 저렇게 불확실한 얘기를 믿고 미리 당겨서 협의를 할 필요가 있는가, 그런 생각이 일단 든다”고 말했다. 한 대표와의 회동 가능성에 대해서는 “요청은 했는데 아직 결정을 통보받지 못했다. (한 대표 측에서) 오후에 다시 연락하자는 연락이 왔다”고 전했다.

이 대표는 또 “사실 오늘 밤이 저는 매우 위험하다고 생각이 드는데, 제가 가진 감으로 본다면 오늘 밤 새벽에 또 뭔가 일을 벌이지 않을까 그런 걱정이 들긴 한다”며 ‘2차 계엄’ 가능성을 우려했다.

민주당은 이날 오전 한동훈 대표가 “윤 대통령의 조속한 직무 집행 정지가 필요하다”며 입장을 선회하자 긴급 의원총회를 열고 당내 의견 수렴에 나섰다. 의원총회에서는 탄핵소추안 표결 시점을 앞당기는 방안도 논의될 전망이다.

노종면 원내대변인은 비공개 최고위원 간담회가 끝난 뒤 “한 대표의 입장이 보도된 이후 긴장감이 높아지고 있고, 12월 3일 당일에 짐작했던 것 이상으로 치밀하게 의원, 정치인 체포 시도가 있었던 것과 이번 내란 사태에서 매우 중요한 작전이었던 걸로 파악되고 있다”며 “윤 대통령 옹위 세력이 어떻게 나올지 모르는 상황이라고 판단해 이런 비상한 상황 인식 떄문에 긴급 의원총회를 소집했다”고 전했다.

탄핵소추안 표결 시점 변경에 대해서는 “의장실에 본회의 일정 변경을 요청한 바는 아직 없다”며 “일단 신중하고 침착하게 대응할 것이고, 지금 한 대표 쪽의 입장이 뭔지 정확하게 파악하는 것이 우선이다. 필요하면 본회의를 앞당기는 방안도 의장실과 협조해서 추진할 수 있지만 아직은 결정된 바 없다”고 밝혔다.

민주당에서는 7일 오후 7시로 예정됐던 표결을 2시간 당겨 오후 5시에 추진하는 방안도 거론된다. 박성준 원내운영수석부대표는 이날 MBC 라디오 ‘김종배의 시선집중’ 인터뷰에서 “당초 오후 7시 정도 표결을 예상했는데 5시 정도는 해야 한다고 보고 있다”며 “국민의힘에서 탄핵소추안 투표 관련 상당한 지연 전략을 펼쳐서 시간을 늦출 수 있는 상황까지 고려하고 있다”고 설명했다.
"""

encoded_text_gpt4o = gpt4o.encode(text)
encoded_text_gpt41 = gpt41.encode(text)

print("gpt-4o 토큰수: ",len(encoded_text_gpt4o))
print("gpt-4.1 토큰수: ",len(encoded_text_gpt41))
